# Supervised ANNs for Mutual Information Investigation
## Thomas Possidente

### Imports and Initializations

In [1]:
# ANN Building and Visualization Imports
import keras
from keras import backend as K
from keras import optimizers, losses
from keras.engine.topology import Layer
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
import tensorflow as tf
from keras.callbacks import LambdaCallback
from keras.utils import to_categorical
import matplotlib.pyplot as plt



# Standard Imports
import pandas as pd
import numpy as np
import math

# Value Inits - Specify as needed
num_inputs = int(5120)  # number of dummy images in set
size = int(16)          # Dimension of each dummy image should be size*size
RF_size = int(2)        # Dimensions of the RF to be analyzed should be RF_size*RF_size
noise = 0.5               # percentage (as decimal) of input values that will be flipped 

# Value Inits - Leave these alone
num_of_RFs = int((size*size) / (RF_size*RF_size))


C:\Users\Tom\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Reading in Data

In [2]:
inputs = pd.read_csv('Pattern_Matrices_Datasets/size16_RF2_noise0.csv')
inputs = inputs.drop('X1', axis = 0) # Taking out col names
inputs = inputs.apply(pd.to_numeric)  # converting to floats

inputs = inputs.values # convert to np ndarray
inputs = inputs.reshape(num_inputs, size, size,1) # reshape to desired dims (5000 examples, 16*16 image, 1 channel)
flattened_inputs = inputs.reshape(num_inputs, size*size)

### Cutting Number of Labels Possible to 256 if Necessary

In [3]:
flattened_inputs = flattened_inputs[0:256]
flattened_inputs = np.repeat(flattened_inputs, 20, 0)
flattened_inputs = np.random.permutation(flattened_inputs)

num_inputs = int(flattened_inputs.shape[0])


### Creating Labels

In [4]:
labels = np.zeros(np.shape(flattened_inputs)[0])
storage = np.zeros(np.shape(flattened_inputs))
count = 0

for i in range(np.shape(flattened_inputs)[0]):
    if(~((flattened_inputs[i] == storage).all(1).any())):
        labels[i] = count
        storage[i] = flattened_inputs[i]
        count += 1
    elif((flattened_inputs[i] == storage).all(1).any()):
        label_index = np.where((flattened_inputs[i] == storage).all(1))
        correct_label = labels[label_index]
        storage[i] = flattened_inputs[i]
        labels[i] = correct_label[0]
        
labels = labels.astype(int)
labels = to_categorical(labels)

    

In [5]:
labels.shape

(5120, 256)

### Adding Noise to Inputs

In [6]:
for n in range(np.shape(flattened_inputs)[0]):
    indices_to_flip = np.random.choice(int(size*size), math.ceil(size*size * noise), replace = False)
    flattened_inputs[n][indices_to_flip] = 1 - flattened_inputs[n][indices_to_flip]

inputs = flattened_inputs.reshape(num_inputs, size, size, 1)


In [7]:
model = Sequential()
model.add(Conv2D(input_shape = (16,16,1), filters=4, kernel_size = RF_size, strides = RF_size, activation = 'relu'))
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dense(256, activation = 'softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 8, 8, 4)           20        
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
Total params: 262,932
Trainable params: 262,932
Non-trainable params: 0
_________________________________________________________________


### Building Network

In [8]:
def run_model():
    
    model = Sequential()
    model.add(Conv2D(input_shape = (16,16,1), filters=4, kernel_size = RF_size, strides = RF_size, activation = 'relu'))
    model.add(Flatten())
    model.add(Dense(512, activation = 'relu'))
    model.add(Dense(256, activation = 'softmax'))
    model.compile(optimizer = optimizers.adam(lr = 0.0005), loss = 'categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(x = inputs, validation_split = 0.25, y = labels, batch_size = 100, epochs = 25, shuffle=False)
    return(history)



In [9]:
storage_raw = np.zeros((30,5))
storage_means = np.zeros((6, 8))
param_RF_size = np.array([2,4,8])
num_inputs = int(5000) 
size = int(16)        
counter = 0

for n in range(5):
    RF_size = int(param_RF_size[n])
    
    val_acc15 = np.zeros(10)
    val_acc99 = np.zeros(10)
    val_loss15 = np.zeros(10)
    val_loss99 = np.zeros(10)
    for i in range(10):
        results = run_model()
        val_acc15[i] = results.history['val_acc'][4]
        val_acc99[i] = results.history['val_acc'][24]
        val_loss15[i] = results.history['val_loss'][4]
        val_loss99[i] = results.history['val_loss'][24]
        one_storage_raw = np.array([RF_size, val_acc15[i], val_acc99[i], val_loss15[i], val_loss99[i]])
        storage_raw[counter] = one_storage_raw
        counter += 1

    acc15 = np.mean(val_acc15)
    acc100 = np.mean(val_acc99)
    loss15 = np.mean(val_loss15)
    loss100 = np.mean(val_loss99)
    acc15_SD = np.std(val_acc15)
    acc100_SD = np.std(val_acc99)
    loss15_SD = np.std(val_loss15)
    loss100_SD = np.std(val_loss99)
    
    one_res = np.array([acc15, acc100, loss15, loss100, acc15_SD, acc100_SD, loss15_SD, loss100_SD])
    
    storage_means[n] = one_res
    
    
#print("acc15 = " + str(np.mean(val_acc15)), "\n",
#      "acc100 = " + str(np.mean(val_acc99)), "\n",
#      "loss15 = " + str(np.mean(val_loss15)), "\n",
#      "loss100 = " + str(np.mean(val_loss99)), "\n",
#      "acc15_SD = " + str(np.std(val_acc15)), "\n",
#      "acc100_SD = " + str(np.std(val_acc99)), "\n",
#      "loss15_SD = " + str(np.std(val_loss15)), "\n",
#      "loss100_SD = " + str(np.std(val_loss99)))

Train on 3840 samples, validate on 1280 samples
Epoch 1/25
3840/3840 [==============================] - 1s 276us/step - loss: 5.5976 - acc: 0.0023 - val_loss: 5.5894 - val_acc: 0.0016
Epoch 2/25
3840/3840 [==============================] - 1s 187us/step - loss: 5.4766 - acc: 0.0138 - val_loss: 5.6130 - val_acc: 0.0031
Epoch 3/25
3840/3840 [==============================] - 1s 201us/step - loss: 5.3803 - acc: 0.0302 - val_loss: 5.6379 - val_acc: 0.0055
Epoch 4/25
3840/3840 [==============================] - 1s 182us/step - loss: 5.2443 - acc: 0.0562 - val_loss: 5.6590 - val_acc: 0.0039
Epoch 5/25
3840/3840 [==============================] - 1s 192us/step - loss: 5.0666 - acc: 0.1102 - val_loss: 5.6817 - val_acc: 0.0023
Epoch 6/25
3840/3840 [==============================] - 1s 177us/step - loss: 4.8495 - acc: 0.1659 - val_loss: 5.7157 - val_acc: 0.0039
Epoch 7/25
3840/3840 [==============================] - 1s 179us/step - loss: 4.5938 - acc: 0.2331 - val_loss: 5.7699 - val_acc: 0.0016


Epoch 10/25
3840/3840 [==============================] - 1s 180us/step - loss: 3.7447 - acc: 0.4122 - val_loss: 6.0023 - val_acc: 0.0031
Epoch 11/25
3840/3840 [==============================] - 1s 170us/step - loss: 3.4278 - acc: 0.4562 - val_loss: 6.1387 - val_acc: 0.0031
Epoch 12/25
3840/3840 [==============================] - 1s 168us/step - loss: 3.1179 - acc: 0.4948 - val_loss: 6.2942 - val_acc: 0.0047
Epoch 13/25
3840/3840 [==============================] - 1s 183us/step - loss: 2.8210 - acc: 0.5417 - val_loss: 6.4681 - val_acc: 0.0047
Epoch 14/25
3840/3840 [==============================] - 1s 172us/step - loss: 2.5418 - acc: 0.5870 - val_loss: 6.6455 - val_acc: 0.0039
Epoch 15/25
3840/3840 [==============================] - 1s 177us/step - loss: 2.2824 - acc: 0.6354 - val_loss: 6.8290 - val_acc: 0.0031
Epoch 16/25
3840/3840 [==============================] - 1s 172us/step - loss: 2.0432 - acc: 0.6799 - val_loss: 7.0163 - val_acc: 0.0031
Epoch 17/25
3840/3840 [==================

3840/3840 [==============================] - 1s 182us/step - loss: 1.5823 - acc: 0.7583 - val_loss: 7.5468 - val_acc: 0.0062
Epoch 20/25
3840/3840 [==============================] - 1s 169us/step - loss: 1.4049 - acc: 0.7979 - val_loss: 7.7421 - val_acc: 0.0070
Epoch 21/25
3840/3840 [==============================] - 1s 180us/step - loss: 1.2423 - acc: 0.8318 - val_loss: 7.9338 - val_acc: 0.0070
Epoch 22/25
3840/3840 [==============================] - 1s 178us/step - loss: 1.0938 - acc: 0.8628 - val_loss: 8.1302 - val_acc: 0.0062
Epoch 23/25
3840/3840 [==============================] - 1s 187us/step - loss: 0.9613 - acc: 0.8935 - val_loss: 8.3184 - val_acc: 0.0062
Epoch 24/25
3840/3840 [==============================] - 1s 194us/step - loss: 0.8382 - acc: 0.9190 - val_loss: 8.4995 - val_acc: 0.0070
Epoch 25/25
3840/3840 [==============================] - 1s 185us/step - loss: 0.7295 - acc: 0.9414 - val_loss: 8.6877 - val_acc: 0.0070
Train on 3840 samples, validate on 1280 samples
Epoch

Epoch 3/25
3840/3840 [==============================] - 1s 190us/step - loss: 5.4129 - acc: 0.0396 - val_loss: 5.6068 - val_acc: 0.0023
Epoch 4/25
3840/3840 [==============================] - 1s 172us/step - loss: 5.3046 - acc: 0.0690 - val_loss: 5.6308 - val_acc: 0.0023
Epoch 5/25
3840/3840 [==============================] - 1s 188us/step - loss: 5.1569 - acc: 0.1029 - val_loss: 5.6484 - val_acc: 0.0039
Epoch 6/25
3840/3840 [==============================] - 1s 191us/step - loss: 4.9702 - acc: 0.1482 - val_loss: 5.6763 - val_acc: 0.0047
Epoch 7/25
3840/3840 [==============================] - 1s 175us/step - loss: 4.7478 - acc: 0.1937 - val_loss: 5.7234 - val_acc: 0.0031
Epoch 8/25
3840/3840 [==============================] - 1s 181us/step - loss: 4.5013 - acc: 0.2375 - val_loss: 5.7910 - val_acc: 0.0039
Epoch 9/25
3840/3840 [==============================] - 1s 172us/step - loss: 4.2403 - acc: 0.2753 - val_loss: 5.8818 - val_acc: 0.0047
Epoch 10/25
3840/3840 [=========================

Epoch 12/25
3840/3840 [==============================] - 1s 166us/step - loss: 3.4541 - acc: 0.3932 - val_loss: 6.2834 - val_acc: 0.0016
Epoch 13/25
3840/3840 [==============================] - 1s 179us/step - loss: 3.1841 - acc: 0.4328 - val_loss: 6.4436 - val_acc: 0.0016
Epoch 14/25
3840/3840 [==============================] - 1s 165us/step - loss: 2.9248 - acc: 0.4747 - val_loss: 6.6098 - val_acc: 7.8125e-04
Epoch 15/25
3840/3840 [==============================] - 1s 169us/step - loss: 2.6784 - acc: 0.5148 - val_loss: 6.7827 - val_acc: 0.0016
Epoch 16/25
3840/3840 [==============================] - 1s 182us/step - loss: 2.4459 - acc: 0.5651 - val_loss: 6.9616 - val_acc: 0.0016
Epoch 17/25
3840/3840 [==============================] - 1s 170us/step - loss: 2.2270 - acc: 0.6089 - val_loss: 7.1430 - val_acc: 0.0016
Epoch 18/25
3840/3840 [==============================] - 1s 181us/step - loss: 2.0207 - acc: 0.6482 - val_loss: 7.3279 - val_acc: 0.0023
Epoch 19/25
3840/3840 [==============

...

Epoch 7/25
3840/3840 [==============================] - 1s 140us/step - loss: 5.3371 - acc: 0.0237 - val_loss: 5.6590 - val_acc: 0.0047
Epoch 8/25
3840/3840 [==============================] - 1s 153us/step - loss: 5.2814 - acc: 0.0260 - val_loss: 5.6814 - val_acc: 0.0039
Epoch 9/25
3840/3840 [==============================] - 1s 141us/step - loss: 5.2274 - acc: 0.0294 - val_loss: 5.7093 - val_acc: 0.0039
Epoch 10/25
3840/3840 [==============================] - 1s 142us/step - loss: 5.1758 - acc: 0.0333 - val_loss: 5.7436 - val_acc: 0.0031
Epoch 11/25
3840/3840 [==============================] - 1s 158us/step - loss: 5.1283 - acc: 0.0370 - val_loss: 5.7814 - val_acc: 0.0031
Epoch 12/25
3840/3840 [==============================] - 1s 142us/step - loss: 5.0849 - acc: 0.0406 - val_loss: 5.8199 - val_acc: 0.0031
Epoch 13/25
3840/3840 [==============================] - 1s 157us/step - loss: 5.0452 - acc: 0.0435 - val_loss: 5.8602 - val_acc: 0.0031
Epoch 14/25
3840/3840 [=====================

Epoch 16/25
3840/3840 [==============================] - 1s 150us/step - loss: 4.9390 - acc: 0.0521 - val_loss: 6.0459 - val_acc: 0.0016
Epoch 17/25
3840/3840 [==============================] - 1s 136us/step - loss: 4.9095 - acc: 0.0544 - val_loss: 6.0836 - val_acc: 0.0016
Epoch 18/25
3840/3840 [==============================] - 1s 137us/step - loss: 4.8814 - acc: 0.0568 - val_loss: 6.1205 - val_acc: 0.0016
Epoch 19/25
3840/3840 [==============================] - 1s 154us/step - loss: 4.8545 - acc: 0.0591 - val_loss: 6.1552 - val_acc: 0.0016
Epoch 20/25
3840/3840 [==============================] - 1s 136us/step - loss: 4.8285 - acc: 0.0609 - val_loss: 6.1868 - val_acc: 0.0016
Epoch 21/25
3840/3840 [==============================] - 1s 140us/step - loss: 4.8030 - acc: 0.0625 - val_loss: 6.2191 - val_acc: 0.0023
Epoch 22/25
3840/3840 [==============================] - 1s 149us/step - loss: 4.7778 - acc: 0.0664 - val_loss: 6.2494 - val_acc: 0.0023
Epoch 23/25
3840/3840 [==================

3840/3840 [==============================] - 1s 145us/step - loss: 4.6796 - acc: 0.0737 - val_loss: 6.3325 - val_acc: 0.0016
Train on 3840 samples, validate on 1280 samples
Epoch 1/25
3840/3840 [==============================] - 1s 377us/step - loss: 5.5478 - acc: 0.0031 - val_loss: 5.5491 - val_acc: 0.0016
Epoch 2/25
3840/3840 [==============================] - 1s 156us/step - loss: 5.5352 - acc: 0.0076 - val_loss: 5.5578 - val_acc: 0.0016
Epoch 3/25
3840/3840 [==============================] - 1s 141us/step - loss: 5.5166 - acc: 0.0107 - val_loss: 5.5756 - val_acc: 7.8125e-04
Epoch 4/25
3840/3840 [==============================] - 1s 143us/step - loss: 5.4881 - acc: 0.0112 - val_loss: 5.6027 - val_acc: 7.8125e-04
Epoch 5/25
3840/3840 [==============================] - 1s 154us/step - loss: 5.4494 - acc: 0.0177 - val_loss: 5.6294 - val_acc: 7.8125e-04
Epoch 6/25
3840/3840 [==============================] - 1s 141us/step - loss: 5.4025 - acc: 0.0195 - val_loss: 5.6474 - val_acc: 7.8125

3840/3840 [==============================] - 1s 148us/step - loss: 5.2878 - acc: 0.0276 - val_loss: 5.6946 - val_acc: 0.0000e+00
Epoch 9/25
3840/3840 [==============================] - 1s 157us/step - loss: 5.2331 - acc: 0.0302 - val_loss: 5.7245 - val_acc: 0.0000e+00
Epoch 10/25
3840/3840 [==============================] - 1s 185us/step - loss: 5.1805 - acc: 0.0367 - val_loss: 5.7603 - val_acc: 7.8125e-04
Epoch 11/25
3840/3840 [==============================] - 1s 169us/step - loss: 5.1313 - acc: 0.0401 - val_loss: 5.8002 - val_acc: 7.8125e-04
Epoch 12/25
3840/3840 [==============================] - 1s 147us/step - loss: 5.0860 - acc: 0.0437 - val_loss: 5.8421 - val_acc: 7.8125e-04
Epoch 13/25
3840/3840 [==============================] - 1s 144us/step - loss: 5.0446 - acc: 0.0458 - val_loss: 5.8839 - val_acc: 7.8125e-04
Epoch 14/25
3840/3840 [==============================] - 1s 156us/step - loss: 5.0067 - acc: 0.0495 - val_loss: 5.9258 - val_acc: 7.8125e-04
Epoch 15/25
3840/3840 [===

IndexError: index 3 is out of bounds for axis 0 with size 3

In [10]:
storage_raw_pd = pd.DataFrame(storage_raw)
storage_raw_pd

,0,1,2,3,4
0,2.0,0.002344,0.002344,5.681668,8.687166
1,2.0,0.000000,0.002344,5.667724,8.456478
2,2.0,0.000781,0.002344,5.646634,8.666628
3,2.0,0.000781,0.002344,5.657588,7.968990
4,2.0,0.003906,0.007031,5.677555,8.687682
5,2.0,0.000781,0.001562,5.658367,8.608605
6,2.0,0.001562,0.000781,5.651743,8.743619
7,2.0,0.003906,0.001562,5.648373,8.516000
8,2.0,0.001562,0.003125,5.676805,8.723695
9,2.0,0.000781,0.003125,5.664969,8.667148


In [11]:
storage_means_pd = pd.DataFrame(storage_means)
storage_means_pd

,0,1,2,3,4,5,6,7
0,0.001641,0.002656,5.663143,8.572601,0.001281,0.001609,0.011984,0.218759
1,0.002266,0.003125,5.644984,7.296426,0.000737,0.001210,0.008907,0.075272
2,0.002031,0.003984,5.627196,6.259628,0.001169,0.001281,0.006355,0.043583
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### TODOS
* Find Image dataset that's binarized (or binarize existing one) and do MI calculations to find max MI. To do this, sample RF sized chunks of an image until stable average MI value is reached
* https://people.cs.umass.edu/~marlin/data.shtml, http://vision.lems.brown.edu/content/available-software-and-databases